In [18]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix, ndcs_to_pixels
from visualization import load_depth
import progressbar
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle
from voxelmaps import camera_to_pointcloud, load_scene_db_data, get_main_image_name, scene_to_pointcloud, scene_to_voxelmap, NoMainImageException, scene_to_voxelmap_with_map, get_main_image
import voxelmaps
import time

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = CONFIG["Images"]["MlDatasetVoxel"]
out_root_directory = r'D:\Projects\GTAVisionExport_postprocessing'


In [3]:
run_id = 6

cur.execute("""SELECT DISTINCT scene_id \
  FROM snapshots \
  WHERE run_id = {} \
  """.format(run_id))

scenes = []
for row in cur:
    res = dict(row)
    scenes.append(res)

image_names = {}

print('There are {} scenes'.format(len(scenes)))


There are 8438 scenes


### functions for each scene

In [16]:
# import time
voxelmaps.MAX_DISTANCE = 30 # in meters, meaning I care only for point up to 20 meters

points_cache = {}

def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])


def convert_tiff(in_directory, out_directory, out_name, name):
    # throws OSError
    out_format = 'jpg'
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    if os.path.exists(outfile):
        return outfile
    infile = os.path.join(in_directory, name)
    im = Image.open(infile)
    im = im.convert(mode="RGB")
    # print("Generating new format for {} to new file {}".format(name, out_name))
    im.save(outfile)
    return outfile

def generate_frustum_points(proj_matrix):
    x_min = -1  # NDC corners
    x_max = 1  # NDC corners
    y_min = -1  # NDC corners
    y_max = 1  # NDC corners
    x_range = 240  # X output size of NN
    y_range = 160  # Y output size of NN
    z_range = 100  # Z output size of NN
    z_meters_min = 1.5
    z_meters_max = voxelmaps.MAX_DISTANCE
    # z min calc
    z_min = proj_matrix @ [1, 1, -z_meters_max, 1]
    z_min = z_min[2] / z_min[3]
    # z max calc
    z_max = proj_matrix @ [1, 1, -z_meters_min, 1]
    z_max = z_max[2] / z_max[3]

    x_bin = (x_max - x_min) / x_range
    y_bin = (y_max - y_min) / y_range
    z_bin = (z_max - z_min) / z_range

    if proj_matrix.tobytes() in points_cache:
        return points_cache[proj_matrix.tobytes()].copy(), x_range, y_range, z_range, z_bin, z_min

    X, Y, Z, W = np.mgrid[x_min:x_max:x_bin, y_min:y_max:y_bin, z_min:z_max:z_bin, 1:2:1]  # W is here as homo coordinate
    positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()])
    
    points_cache[proj_matrix.tobytes()] = positions
    return positions, x_range, y_range, z_range, z_bin, z_min

def convert_ndc_pointcloud_to_bool_grid(x_range, y_range, z_range, occupied_ndc_positions, z_bin, z_min):
    # now I create x X y X z grid with 0s and 1s as grid
    # so now I have data in pointcloud. And I need to convert these NDC values
    # into indices, so x:[-1, 1] into [0, 239], y:[-1, 1] to [0, 159], 
    # and z:[z_min, z_max] into [0, 99]
    voxelmap_ndc_grid = np.zeros((x_range, y_range, z_range), dtype=np.bool)
    vecs = ndcs_to_pixels(occupied_ndc_positions[0:2, :], (y_range, x_range))
    vec_y = vecs[0, :]
    vec_x = vecs[1, :]
    vec_z = ((occupied_ndc_positions[2, :] - z_min) / z_bin).astype(np.int32)
    voxelmap_ndc_grid[vec_x, vec_y, vec_z] = 1
    return voxelmap_ndc_grid

def convert_scene_to_ndc_voxelmap(cameras):
    # this method is just fucking slow, because of scene_to_voxelmap_with_map
    # here I generate points cuboid which will be output of NN.
    # these points will be transferred into camera view frustum
    # for this frustum, I take voxel values of calculated voxelmap
    # I use these voxel values as values for cuboid in NDC, the NN output
    start = time.time()

    voxels, values, map_voxel_size, map_obj = scene_to_voxelmap_with_map(cameras)

    end = time.time()
    print('scene to whole voxelmap:', end - start)
    start = time.time()

    cam = get_main_image(cameras)
    proj_matrix = cam['proj_matrix']
    view_matrix = cam['view_matrix']

    end = time.time()
    print('getting main cam and stuff:', end - start)
    start = time.time()

    positions, x_range, y_range, z_range, z_bin, z_min = generate_frustum_points(proj_matrix)

    end = time.time()
    print('generating frustum points:', end - start)
    start = time.time()

    points_view = ndc_to_view(positions, proj_matrix)
    points_world = view_to_world(points_view, view_matrix)

    end = time.time()
    print('obtaining world frustum:', end - start)
    start = time.time()

    # here I find corresponding voxels for generated points, by obtaining voxelmap reference
    voxel_values = map_obj.get_voxels(points_world[0:3, :], np.zeros((points_world.shape[1], 1)))
    occupied_ndc_positions = positions[:, voxel_values > 0]

    end = time.time()
    print('obtaining voxelmap in NDC:', end - start)
    start = time.time()
    # this converts ndc pointcloud to bool grid
    occupied_ndc_grid = convert_ndc_pointcloud_to_bool_grid(x_range, y_range, z_range, occupied_ndc_positions, z_bin, z_min)

    end = time.time()
    print('pointcloud to bool grid:', end - start)

    return occupied_ndc_grid

    
def convert_scene_to_img_and_voxelmap(in_directory, out_directory, scene_id):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    # start = time.time()
    cameras = load_scene_db_data(scene_id)
    try:
        image_name = get_main_image_name(cameras)
        rgb_outfile = convert_tiff(in_directory, out_directory, image_name, image_name+'.tiff')
        outfile = os.path.join(out_directory, "{}.bin".format(image_name))

        if os.path.exists(outfile):
            return
        
        #start = time.time()
        occupied_ndc_positions = convert_scene_to_ndc_voxelmap(cameras)
        #end = time.time()
        #print('time to do whole scene to voxelmap:', end - start)
        #start = time.time()
        
        occupied_ndc_positions.tofile(outfile)
        #end = time.time()
        #print('time to save:', end - start)
        
        image_names[scene_id] = (rgb_outfile, outfile)
    except (OSError, NoMainImageException) as e:
        print('No main image for scene {}, skipping.'.format(scene_id))


### actually running the extraction

In [5]:
workers = 10

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(scenes)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(convert_scene_to_img_and_voxelmap)(in_directory, out_directory, i['scene_id']) for i in scenes)


  0% 11 |                                                           |  71.9 B/sC:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in greater


time to do whole scene to voxelmap: 45.40229940414429


  0% 16 |                                                           |   0.3 B/s

time to do whole scene to voxelmap:time to save:time to do whole scene to voxelmap:  57.48637008666992 65.94342851638794
3.898021936416626



  0% 17 |                                                           |   0.2 B/s

...

time to save: 0.13300204277038574


  0% 35 |                                                           |   0.2 B/s

time to do whole scene to voxelmap: 97.12862610816956
time to save: 0.1510014533996582


KeyboardInterrupt: 

time to do whole scene to voxelmap: 70.37145781517029
time to do whole scene to voxelmap:time to do whole scene to voxelmap:time to save:   2.999011516571045103.56367254257202104.13567876815796


time to save: 0.08500075340270996
time to save: 0.0989999771118164
time to do whole scene to voxelmap: 105.03668332099915
time to save:time to do whole scene to voxelmap:  0.1250023841857910269.89045524597168

time to save: 2.0770153999328613
time to do whole scene to voxelmap: 109.95371437072754
time to save: 0.04900217056274414
time to do whole scene to voxelmap: 108.96970725059509
time to save: 4.885041952133179
time to do whole scene to voxelmap: 117.48276448249817
time to save: 0.10500502586364746
time to do whole scene to voxelmap: 117.47276282310486
time to save: 0.21500539779663086
time to do whole scene to voxelmap: 118.4377670288086
time to save: 0.04399919509887695


### generating filenames list

In [ ]:
filenames = []
for scene_id, (rgb_outfile, outfile) in image_names.items():
    rgb_name = os.path.relpath(os.path.join(out_directory, rgb_outfile), start=out_root_directory)
    rgb_name = rgb_name.replace('\\', '/')
    voxelmap_name = os.path.relpath(os.path.join(out_directory, outfile), start=out_root_directory)
    voxelmap_name = depth_name.replace('\\', '/')
    filenames.append([rgb_name, depth_name])

    # r je abych nemusel psát zpětná lomítka
with open(os.path.join(out_root_directory, 'whole-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(filenames)

random.shuffle(filenames)

train_ratio = 0.8
train_threshold = int(train_ratio * len(image_names))
train_filenames = filenames[:train_threshold]
test_filenames = filenames[train_threshold:]

with open(os.path.join(out_root_directory, 'train-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(train_filenames)
with open(os.path.join(out_root_directory, 'test-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(test_filenames)


### running extraction for one scene

In [19]:
convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[0]['scene_id'])

  0% 41 |                                                           |  44.6 s/B

scene to whole voxelmap: 10.923073530197144
getting main cam and stuff: 0.0
generating frustum points: 0.03501081466674805
obtaining world frustum: 0.23198962211608887
obtaining voxelmap in NDC: 0.26500391960144043
pointcloud to bool grid: 0.011000394821166992


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in greater


In [20]:
# just some timing experiments
for i in range(5):
    convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[i]['scene_id'])

  0% 42 |                                                           |  44.1 s/B

scene to whole voxelmap: 11.031072616577148
getting main cam and stuff: 0.0009903907775878906
generating frustum points: 0.03500843048095703
obtaining world frustum: 0.23499274253845215


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in greater
  0% 43 |                                                           |  43.4 s/B

obtaining voxelmap in NDC: 0.250011682510376
pointcloud to bool grid: 0.01300048828125
scene to whole voxelmap: 11.20207691192627
getting main cam and stuff: 0.0
generating frustum points: 0.03500795364379883
obtaining world frustum: 0.24100065231323242


  0% 44 |                                                           |  42.7 s/B

obtaining voxelmap in NDC: 0.2460010051727295
pointcloud to bool grid: 0.011999845504760742
scene to whole voxelmap: 11.562061071395874
getting main cam and stuff: 0.0
generating frustum points: 0.03501558303833008
obtaining world frustum: 0.23299527168273926


  0% 45 |                                                           |  42.0 s/B

obtaining voxelmap in NDC: 0.26199936866760254
pointcloud to bool grid: 0.007999897003173828
scene to whole voxelmap: 9.643060684204102
getting main cam and stuff: 0.0
generating frustum points: 0.03500175476074219
obtaining world frustum: 0.2350025177001953


  0% 46 |                                                           |  41.3 s/B

obtaining voxelmap in NDC: 0.2590045928955078
pointcloud to bool grid: 0.010005712509155273
scene to whole voxelmap: 8.779057502746582
getting main cam and stuff: 0.0
generating frustum points: 0.03400087356567383
obtaining world frustum: 0.23399925231933594
obtaining voxelmap in NDC: 0.25101304054260254
pointcloud to bool grid: 0.008988618850708008


In [ ]:
prev = None
for i in range(5):
    cameras = load_scene_db_data(scenes[i]['scene_id'])
    cam = get_main_image(cameras)
    proj_matrix = cam['proj_matrix']
    if prev is not None:
        print(np.array_equal(prev, proj_matrix))
    prev = proj_matrix

In [ ]:
prev